In [52]:
# example_meta_data_reader.ipynb
# Authors: Stephan Meighen-Berger
# Shows how to deal with the output files

In [53]:
%matplotlib qt
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import h5py


In [55]:
example_set = ak.from_parquet('./output/custom_1337_meta_data.parquet')

In [56]:
example_set.fields

['event_id', 'mc_truth', 'primary_lepton_1', 'primary_hadron_1']

In [57]:
example_set.mc_truth.fields

['injection_type',
 'injection_interaction_type',
 'injection_zenith',
 'injection_azimuth',
 'injection_bjorkenx',
 'injection_bjorkeny',
 'injection_position_x',
 'injection_position_y',
 'injection_position_z',
 'injection_column_depth',
 'primary_lepton_1_type',
 'primary_hadron_1_type',
 'primary_lepton_1_position_x',
 'primary_lepton_1_position_y',
 'primary_lepton_1_position_z',
 'primary_hadron_1_position_x',
 'primary_hadron_1_position_y',
 'primary_hadron_1_position_z',
 'primary_lepton_1_direction_theta',
 'primary_lepton_1_direction_phi',
 'primary_hadron_1_direction_theta',
 'primary_hadron_1_direction_phi',
 'primary_lepton_1_energy',
 'primary_hadron_1_energy',
 'total_energy']

In [60]:
for field in example_set.mc_truth.fields:
    print(field)
    print(example_set.mc_truth[field])

injection_type
[14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
injection_interaction_type
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
injection_zenith
[2.35, 1.41, 2.11, 1.64, 1.51, 2.81, 1.76, 1.79, 2.62, 1.59]
injection_azimuth
[4.93, 5.82, 1.71, 3.94, 3.15, 5.31, 4.33, 2.9, 5.65, 3.73]
injection_bjorkenx
[0.293, 0.219, 0.0702, 0.148, 0.404, 0.0713, 0.322, 0.151, 0.388, 0.403]
injection_bjorkeny
[0.293, 0.219, 0.0702, 0.148, 0.404, 0.0713, 0.322, 0.151, 0.388, 0.403]
injection_position_x
[-161, -163, 1.62, 66.2, 61.6, 148, 264, 236, -161, -174]
injection_position_y
[200, -188, 107, 272, -14, 242, -61.6, 128, -54.6, 229]
injection_position_z
[111, 17, -41, 66.6, 185, 29.8, 46.7, 112, 47.1, 22]
injection_column_depth
[4.68e+04, 3.34e+04, 6.4e+04, 4.89e+04, ... 2.52e+04, 4.52e+04, 4.25e+04, 1.61e+04]
primary_lepton_1_type
[13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
primary_hadron_1_type
[-2000001006, -2000001006, -2000001006, ... -2000001006, -2000001006, -2000001006]
primary_lepton_1_position_x
[-161, -163, 

In [ ]:
example_set.mc_truth.lepton_position_x

<Array [223, 159, -11.8, ... -171, -290, -82.1] type='20 * float64'>

In [ ]:
example_set.event_id

<Array [0, 1, 2, 3, 4, ... 15, 16, 17, 18, 19] type='20 * int64'>

In [ ]:
# The events are split into three parts:
#   - lepton: the light from the generated lepton
#   - hadron: the light produced from the associated hadronic cascade (at production point)
#   - total: the light from both parts
# Each of these fields is formatted the same way
example_set.total.fields

['sensor_id',
 'sensor_pos_x',
 'sensor_pos_y',
 'sensor_pos_z',
 'sensor_string_id',
 't']

In [ ]:
# Example for one event
event_id = 1
# The sensors hit
print("Total number of hits:")
print(len(example_set.total.sensor_id[event_id]))
print('The hit sensors: ')
print(example_set.total.sensor_id[event_id])
print('Their associated strings')
print(example_set.total.sensor_string_id[event_id])
print('The hit times')
print(example_set.total.t[event_id])

Total number of hits:
34
The hit sensors: 
[83, 92, 106, 412, 636, 675, 942, 1029, ... 854, 1142, 1315, 1885, 2991, 3276, 3277]
Their associated strings
[3, 4, 4, 18, 28, 30, 42, 46, 58, 62, ... 143, 17, 38, 51, 59, 85, 135, 148, 148]
The hit times
[1.17e+03, 1.09e+03, 726, 882, 798, 740, 1.02e+03, ... 583, 578, 662, 360, 389, 369]


In [ ]:
def paqruet_plotting(
        data, e_id,
        brightest_event=False,
        channel='lepton',
        show_lepton=True,
        scaling=20
    ):
    try:
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        hit_counts = []
        if brightest_event:
            for event in data[channel].sensor_pos_x:
                hit_counts.append(len(event))
            hit_counts = np.array(hit_counts)
            event_id = np.argmax(hit_counts)
            print('The brightest event has the id %d' % event_id)
            print('The energy of the lepton is %.1f' % data.mc_truth.lepton_energy[event_id])
        else:
            event_id = e_id
        sensor_comb = np.array([
            [data[channel].sensor_pos_x[event_id][i],
            data[channel].sensor_pos_y[event_id][i],
            data[channel].sensor_pos_z[event_id][i]
            ]
            for i in range(len(data[channel].sensor_pos_x[event_id]))
        ])
        hit_doms, hit_counts = np.unique(sensor_comb, axis=0, return_counts=True)
        ax.scatter(
            hit_doms[:, 0],
            hit_doms[:, 1],
            hit_doms[:, 2],
            s=(
                hit_counts / max(hit_counts) * scaling
            ),
            color='r', alpha=0.5)
        if show_lepton:
            ax.scatter(
                data.lepton.loss_pos_x[event_id],
                data.lepton.loss_pos_y[event_id],
                data.lepton.loss_pos_z[event_id],
                s=(
                    data.lepton.loss_n_photons[event_id] /
                    max(data.lepton.loss_n_photons[event_id]) *
                    scaling
                ),
                color='blue', alpha=0.5)
        ax.set_xlim(min(hit_doms[:, 0]), max(hit_doms[:, 0]))
        ax.set_ylim(min(hit_doms[:, 1]), max(hit_doms[:, 1]))
        ax.set_zlim(min(hit_doms[:, 2]), max(hit_doms[:, 2]))
        ax.set_xlabel('X [m]')
        ax.set_ylabel('Y [m]')
        ax.set_zlabel('Z [m]')
        plt.title(
            'Event %d with pdg id = %d and energy %.1f GeV' % (
                event_id,
                data.mc_truth.lepton_type[event_id],
                data.mc_truth.lepton_energy[event_id]
                )
        )
    except:
        print('No hits found!')

In [ ]:
paqruet_plotting(
    example_set,
    e_id=4,
    brightest_event=True,
    channel='total',
    show_lepton=True,
    scaling=100
)

The brightest event has the id 6
The energy of the lepton is 40.4
